# EMT Simulation of CIGRE MV with PV Inverters

### Run simulation

In [ ]:
%%bash
TOP=${TOP:-$(git rev-parse --show-toplevel)}
PATH=${TOP}/build/Examples/Cxx

EMT_CIGRE_MV_withDG --timestep=1e-3 --duration=3 ${TOP}/build/_deps/cim-data-src/CIGRE_MV/NEPLAN/CIGRE_MV_no_tapchanger_noLoad1_LeftFeeder_With_LoadFlow_Results/*.xml

## Powerflow

### Read results

In [ ]:
time_step = 0.001
num_steps_window = int(0.02/time_step)

from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import matplotlib.pyplot as plt
import re

# %matplotlib widget

modelName = 'EMT_CIGRE_MV_withDG_Powerflow'
path = 'logs/' + modelName + '/'
dpsim_result_file = path + modelName + '.csv'

ts_dpsim_powerflow = read_timeseries_csv(dpsim_result_file)

### Plot results

In [ ]:
plt.figure(figsize=(12,8))
for ts_name, ts_obj  in ts_dpsim_powerflow.items():
    if ts_name != 'N0.V':
        plt.plot(ts_obj.time, ts_obj.abs().values, label=ts_name)
plt.legend()
plt.show()

### Node voltages

In [ ]:
for ts_name, ts_obj  in ts_dpsim_powerflow.items():
    print(ts_name + ': ' + str(ts_obj.abs().values[0]) + ', ' + str(ts_obj.phase().values[0]))

## EMT

### Read results

In [ ]:
modelName = 'EMT_CIGRE_MV_withDG'
path = 'logs/' + modelName + '/'
dpsim_result_file = path + modelName + '.csv'

ts_dpsim = read_timeseries_csv(dpsim_result_file)

### Plot results

### All node voltages

In [ ]:
plt.figure(figsize=(12,12))
subplot_number = 1
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name[-3:] == 'V_0' and ts_name[:-2] != 'N0.V':
        plt.subplot(4,3,subplot_number)
        subplot_number += 1
        plt.plot(ts_obj.time, ts_obj.values, label=ts_name)
        plt.legend(loc='upper right')
        plt.xlim([-0.03,1])
plt.show()

## Voltages at first timestep

In [ ]:
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name[-3:] == 'V_0':
        print(ts_name + ': ' + str(ts_obj.values[0]))

## Maximum voltage in last 20ms window

In [ ]:
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name[-3:] == 'V_0':
        print(ts_name + ': ' + str(max(ts_obj.values[-num_steps_window:])))

## PV inverter

### States - Powers

In [ ]:
plt.figure(figsize=(12,6))
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name[-7:]=='state_p' or ts_name[-7:]=='state_q':
        plt.plot(ts_obj.time, ts_obj.abs().values, label=ts_name)
plt.legend()
plt.show()

### States - Others

In [ ]:
plt.figure(figsize=(12,6))
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name[-7:]!='state_p' and ts_name[-7:]!='state_q' and  'state' in ts_name:
        plt.plot(ts_obj.time, ts_obj.abs().values, label=ts_name)
plt.legend()
plt.show()

### States at first timestep

In [ ]:
for ts_name, ts_obj  in ts_dpsim.items():
    if 'state' in ts_name:
        print(ts_name + ': ' + str(ts_obj.abs().values[0]) + ', ' + str(ts_obj.phase().values[0]))

### States at last timestep

In [ ]:
for ts_name, ts_obj  in ts_dpsim.items():
    if 'state' in ts_name:
        print(ts_name + ': ' + str(ts_obj.abs().values[-1]) + ', ' + str(ts_obj.phase().values[-1]))

### Plot inputs

In [ ]:
plt.figure(figsize=(12,6))
for ts_name, ts_obj  in ts_dpsim.items():
    if 'input' in ts_name:
        plt.plot(ts_obj.time, ts_obj.abs().values, label=ts_name)
plt.legend()
plt.show()

### Inputs at first timestep

In [ ]:
for ts_name, ts_obj  in ts_dpsim.items():
    if 'input' in ts_name:
        print(ts_name + ': ' + str(ts_obj.abs().values[0]) + ', ' + str(ts_obj.phase().values[0]))

### Inputs at last timestep

In [ ]:
for ts_name, ts_obj  in ts_dpsim.items():
    if 'input' in ts_name:
        print(ts_name + ': ' + str(ts_obj.abs().values[-1]) + ', ' + str(ts_obj.phase().values[-1]))

### Plot outputs

In [ ]:
plt.figure(figsize=(12,6))
for ts_name, ts_obj  in ts_dpsim.items():
    if 'output' in ts_name:
        plt.plot(ts_obj.time, ts_obj.abs().values, label=ts_name)
plt.legend()
plt.show()

### Outputs at first timestep

In [ ]:
for ts_name, ts_obj  in ts_dpsim.items():
    if 'output' in ts_name:
        print(ts_name + ': ' + str(ts_obj.abs().values[0]) + ', ' + str(ts_obj.phase().values[0]))

### Outputs at last timestep

In [ ]:
for ts_name, ts_obj  in ts_dpsim.items():
    if 'output' in ts_name:
        print(ts_name + ': ' + str(ts_obj.abs().values[-1]) + ', ' + str(ts_obj.phase().values[-1]))

### Plot interface vars

In [ ]:
plt.figure(figsize=(12,6))
subplot_number = 1   
for ts_name, ts_obj  in ts_dpsim.items():
    if 'intf' in ts_name:
        plt.subplot(2,3,subplot_number)
        subplot_number += 1
        plt.plot(ts_obj.time, ts_obj.values, label=ts_name)
        plt.legend(loc='upper right')
plt.show()

## Plot derived power

In [ ]:
pv_N11_p_inst_intf = ts_dpsim['pv_N11_v_intf_0'].values*ts_dpsim['pv_N11_i_intf_0'].values+ts_dpsim['pv_N11_v_intf_1'].values*ts_dpsim['pv_N11_i_intf_1'].values+ts_dpsim['pv_N11_v_intf_2'].values*ts_dpsim['pv_N11_i_intf_2'].values
plt.figure(figsize=(12,6))
plt.plot(ts_dpsim['pv_N11_v_intf_0'].time, pv_N11_p_inst_intf, label='pv_N11_p_inst_intf')
plt.legend()
plt.show()


### Interface vars at first timestep

In [ ]:
for ts_name, ts_obj  in ts_dpsim.items():
    if 'intf' in ts_name:
        print(ts_name + ': ' + str(ts_obj.values[0]))
print('pv_N11_p_inst_intf' + ': ' + str(pv_N11_p_inst_intf[0]))

### Interface vars at last timestep

In [ ]:
for ts_name, ts_obj  in ts_dpsim.items():
    if 'intf' in ts_name:
        print(ts_name + ': ' + str(ts_obj.values[-1]))
print('pv_N11_p_inst_intf' + ': ' + str(pv_N11_p_inst_intf[-1:]))